<a href="https://colab.research.google.com/github/YeonJaeHong/ESAA_homeworks/blob/main/10_2_%EA%B3%BC%EC%A0%9C_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬 머신러닝 (147-175)
03. 평가


머신러닝은 데이터 가공/ 변환, 모델 학습/예측, 그리고 평가의 프로세스로 구성된다.
> 성능 평가 지표는 일반적으로 메딜이 분류냐 회귀냐에 따라 여러 종류로 나뉨.
>>회귀의 경우 대부분 실제 값과 예측값의 오차 평균값에 기반
>>>분류의 성능 평가 지표


*   정확도
*   오차행렬


*   정밀도
*   재현율
>>>> 분류는 결정 클래스 값 종류 유형에 따라 긍정/ 부정 같은 2개의 결괏값만을 가지는 이진 분류/ 여러개의 결정 클래스 값을 가지는 멀티 분류로 나뉠 수 있음






01. accurcy (정확도) = 예측결과가 동일한 데이터 건수/ 전체 예측 데이터 건수

In [58]:
import pandas as pd
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  #fit() 메서드는 아무것도 학습 하지 않음.
  def fit(self,X,y=None):
    pass
 #predict() 메서드는 단순히 sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
  def predict(self,X):
   pred= np.zeros((X.shape[0],1))
   for i in range(X.shape[0]):
     if X['Sex'].iloc[i]==1:
       pred[i]=0
     else:
       pred[i]=1
     return pred

In [80]:
titanic_df = pd.read_csv('/content/titanic_train (2).csv')
print(titanic_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [83]:
#타이타닉 생존자 예측 수행
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
  return df

def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin', 'Sex', 'Embarked']
  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)

  return df

#원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('/content/titanic_train (2).csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df,
                                                  test_size=0.2, random_state=0)

#위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.6089


불균형한 데이터 세트에서의 모델 성능 판단: MNIST 데이터 셋을 사용해서


*   MNIST 데이터 셋을 전체 데이터 10%만 TRUE, 90%가 FALSE인 데이터로 변형
*   정확도 측정



In [84]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


In [85]:
class MyFakeClassifier(BaseEstimator):
  def fit(self,X,Y):
    pass

In [88]:
#입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
def predict(self,X):
  return np.zeros((len(x),1),dtype=bool)
#사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits=load_digits()
#digits 번호가 7번이면 TRUE 이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환.
y= (digits.target==7).astype(int)
X_train,X_test,y_train,y_test=train_test_split(digits.data,y,random_state=11)


In [89]:
#y_test 데이터 분포도 확인, MyFakeClassifier를 이용해 예측, 평가
print('레이블 테스트 세트 크기:', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))


레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


AttributeError: ignored

02. 오차행렬
이진 분류에서 성능 지표 잘 활용되는 오차행렬은 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표. 즉, 이진분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표

: 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표 4분면 행렬에서 실제 레이블 클래스 값과 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지 나타냄 TN, FP, FN, TP 형태로 오차 행렬

TN : 예측값을 Negative값 0으로 예측, 실제값 Negative 값 0
FP : 예측값을 Positive값 1으로 예측, 실제값 Negative 값 0
FN : 예측값을 Negative값 0으로 예측, 실제값 Positive 값 1
TP : 예측값을 Positive값 1으로 예측, 실제값 Positive 값 1

In [90]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

NameError: ignored

3. 정밀도와 재현율
정밀도(양성 예측도) = TP/(FP+TP)
예측을 positive로 한 대상 중 예측과 실제 값이 positive로 일치한 데이터의 비율
재현율(민감도, TPR) = TP/(FN+TP)
실제 값이 positive인 대상 중 예측과 실제 값이 positive로 일치한 데이터의 비율
재현율이 중요 지표인 경우 : 실제 positive 양성 데이터를 negative로 잘못 판단하게 되면 업무상 큰 영향 발생하는 경우 ex) 암 판단, 금융 사기 적발 모델

정밀도가 더 중요 지표인 경우 ex) 스팸메일

재현율과 정밀도는 서로 보완적인 지표로 분류의 성능 평가 -> 재현율과 정밀도 모두 높은 수치 얻는게 좋음

In [91]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차행렬')
  print(confusion)
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))


+ 추가적인 내용 print 할때 왜 '정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))
     이런식으로 쓰냐?


>name= 'Kim'

>>> print('Hello, {}'.format(name))
Hello, Kim
>>> v=2.34567
>>> print('{:1f}'.format(v))
2.345670
>>> print('{:3f}'.format(v))
2.345670
>>> print('{:.3f}'.format(v))
2.346
>>> print('{:.2f}'.format(v))
2.35
>>> per = 99.82828282
>>> print('{:.2f}'.format(per))
99.83



> python의 출력 방식으로,
'{}'.format(출력할것) 이 방식이고,
>> 여기서 '{:.4f}'.format(0.9678)
이런식으로 넣어주면 :.4f 에서 .4는 4번째 자리수로, f는 float 즉, 실수를 말하는 것으로 사용되는 것이다. 그리고 위에서 다루는 {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall)) 와 같은 것은, 여러개를 출력해야 하는 상황이기에, 앞에 0,1,2와 같은 수가 붙고, 0은accuracy, 1은 precision 2는 recall에 할당되어 그 순서로 출력이 되는 것이다. 출력하는 순서를 바꾸고 싶으면 이 수를 바꾸어주면 된다.





03. 정밀도와 재현율


> 정밀도와 재현율은 positive 데이터 세트의 예측 성능에 좀더 초점을 맞춘 평가 지표로 앞엇 만든 myfakeclassifier는 positive로 예측한 tp 값이 하나도 없었기 때문에 재현율의 값이 모두 -이다.




> 정밀도(양성 예측도) = TP/(FP+TP)
예측을 positive로 한 대상 중 예측과 실제 값이 positive로 일치한 데이터의 비율
재현율(민감도, TPR) = TP/(FN+TP)
실제 값이 positive인 대상 중 예측과 실제 값이 positive로 일치한 데이터의 비율

>>재현율이 중요 지표인 경우 : 실제 positive 양성 데이터를 negative로 잘못 판단하게 되면 업무상 큰 영향 발생하는 경우 ex) 암 판단, 금융 사기 적발 모델

정밀도가 더 중요 지표인 경우 ex) 스팸메일

재현율과 정밀도는 서로 보완적인 지표로 분류의 성능 평가 -> 재현율과 정밀도 모두 높은 수치 얻는게 좋음

get_clf_eval() 함수만들기
>> confusionmatrix, accuracy, precision, recall등의 평가를 한번에 호출

In [92]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0:,4f}, 정밀도: {1:.4f}, 재현율:{2:,4f}'.format(accuracy, precision, recall))

In [93]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공, 학습/데이터/테스트 데이터 분할.
titanic_df = pd.read_csv('/content/titanic_train (2).csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df =titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.20, random_state=11)

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

ValueError: ignored

로지스틱 회귀 기반으로 타이타닉 생존자를 예측하고 confusion matrix, accuracy, precision, recall을 수행한것

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# 원본 데이터를 재로딩, 데이터 가공, 학습/데이터/테스트 데이터 분할.
titanic_df = pd.read_csv('/content/titanic_train (2).csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df =titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

import pandas as pd
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  #fit() 메서드는 아무것도 학습 하지 않음.
  def fit(self,X,y=None):
    pass
 #predict() 메서드는 단순히 sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
  def predict(self,X):
   pred= np.zeros((X.shape[0],1))
   for i in range(X.shape[0]):
     if X['Sex'].iloc[i]==1:
       pred[i]=0
     else:
       pred[i]=1
     return pred

def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
  return df

def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin', 'Sex', 'Embarked']
  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)


lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

ValueError: ignored

정밀도/ 재현율 트레이드 오프
> 정밀도와 재현율은 상호 보완적인 평가 지표
>>사이킷런의 분류 알고리즘은 예측 데이터가 특정 레이블에 속하는지를 계산하기 위해서 개별 레이블렬로 결정확률을 구한후, 예측 확률이 큰 레이블값으로 예측한다. ex) 특정 데이터가 0이 될 확률이 10%, 1이 될 확률 90%이면 최종 예측은 더 큰 확률을 가진 1로 예측한다.

logisticRegression 객체의 predict_prob() method의 반환값과 predict() method의 값 비교

In [95]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print('pred_proba()결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

# 예측 확률 array 와 예측 결과값 array 를 concatenate 하여 예측 확률과 결과값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1, 1)], axis=1)
print('두개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n',pred_proba_result[:3])

AttributeError: ignored

더 큰 결과값으로는 predict() 예측값이 나왔다.

사이킷런의 predict()는 predict_proba() method가 반환하는 확률 값을 가진 ndarray 에서 정해진 임계값을 만족하는 ndarray의 칼럼위치를 최종 예측 클래스로 결정함

binarizer 클래스를 객체로 생성. (treshold변수를 특정 값으로 설정하고)
ndarray의 값을 지정된 threshold 보다 같거나 작으면 0값으로, 크면 1값으로 반환

In [77]:
from sklearn.preprocessing import Binarizer

X = [[ 1, -1,  2],
     [ 2,  0,  0],
     [ 0,  1.1, 1.2]]

# X의 개별 원소들이 threshold값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [96]:

from sklearn.preprocessing import Binarizer

# Binarizer의 threshold 설정값, 분류 결정 임곗값임
custom_threshold = 0.5

# predict_proba( ) 반환값의 두번째 컬럼 , 즉 Positive 클래스 컬럼 하나만 추출하여 Binarizer를 적용
pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)


NameError: ignored

In [98]:

#테스트를 수행할 모든 임곗값을 리스트 객체로 저장
thresholds = [0.4, 0.45, 0.50, 0.55, 0.60]

def get_eval_by_threshold(y_test, pred_proba_c1, threshold):
  # thresholds list 객체 내의 값을 차례로 iteration 하면서 evaluation 수행
  for custom_threshold in thresholds:
    binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
    print('임곗값:',custom_threshold)
    get_clf_eval(y_test , custom_predict)

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

NameError: ignored

In [99]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

def precision_recall_curve_plot(y_test , pred_proba_c1):
  #threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출.
    precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)

  #X축을 threshold값으로 Y축은 정밀도, 재현율 값으로 각각 plot수행, 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')

  #threshold 값 X 축의 scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))

  # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

precision_recall_curve_plot( y_test, lr_clf.predict_proba(X_test)[:, 1] )

AttributeError: ignored

f1 SCORE
:정밀도와 재현율을 결합한 지표
어느 한쪽으로 치우쳐지지 않았을때 상대적으로 높은 값을 가지게 된다.

In [101]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test , pred)
print('F1 스코어: {0:.4f}'.format(f1))

NameError: ignored

In [102]:
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    #F1 스코어 추가
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    #f1 score print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}'.format(accuracy, precision, recall, f1))

thresholds = [0.4 , 0.45 , 0.50 , 0.55 , 0.60]
pred_proba = lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

AttributeError: ignored

In [103]:
from sklearn.metrics import roc_curve
#레이블 값이 1일 때의 예측 확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:, 1]

fprs , tprs , thresholds = roc_curve(y_test, pred_proba_class1)

#반환된 임계값 배열 로우가 47건이므로 샘플로 10건만 추출하되, 임곗값을 5step으로 추출
#threshold[0]은 max(예측확률)+1로 임의 설정됨. 이를 제외하기 위해 np.arange는 1부터 시작
thr_index = np.arange(1, thresholds.shape[0], 5)
print('샘플 추출을 위한 임곗값 배열의 index:', thr_index)
print('샘플 index로 추출한 임곗값: ', np.round(thresholds[thr_index], 2))

#5 step 단위로 추출된 임계값에 따른 FPR, TPR 값
print('샘플 임곗값별 FPR: ', np.round(fprs[thr_index], 3))
print('샘플 임곗값별 TPR: ', np.round(tprs[thr_index], 3))

AttributeError: ignored

05. ROC 곡선과 AUC


In [104]:
from sklearn.metrics import roc_curve

# 레이블 값이 1일 때의 예측 확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:, 1]

fprs, tprs, thresholds = roc_curve(y_test, pred_proba_class1)
# 반환된 임곗값 배열 로우가 47건이므로 샘플로 10건만 추출하되, 임곗값을 5 Step으로 추출
# thresholds[0]은 max(예측확률)+1로 임의 설정굄. 이를 제외하기 윟 np.arange는 1부터 시작
thr_index = np.arange(1, thresholds.shape[0], 5)
print('샘플 추출을 위한 임곗값 배열의 index 10개:', thr_index)
print('샘플용 10개의 임곗값:', np.round(thresholds[thr_index], 2))

# 5 step 단위로 추출된 임곗값에 따른 FPR, TPR 값
print('샘플 임곗값별 FPR:', np.round(fprs[thr_index], 3))
print('샘플 임곗값별 TPR:', np.round(tprs[thr_index], 3))

AttributeError: ignored

In [105]:
def roc_curve_plot(y_test, pred_proba_c1):
  # 임곗값에 따른 FPR, TPR 값을 반환받음
  fprs, tprs, thresholds = roc_curve(y_test, pred_proba_c1)
  # ROC 곡선을 그래프 곡선으로 그림
  plt.plot(fprs, tprs, label='ROC')
  # 가운데 대각선 직선을 그림
  plt.plot([0, 1], [0, 1], 'k--', label='Random')

  # FPR X축의 Scale을 0.1 단위로 변경. X, Y축 명 설정 등
  start, end = plt.xlim()
  plt.xticks(np.round(np.arange(start, end, 0.1), 2))
  plt.xlim(0, 1); plt.ylim(0, 1)
  plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
  plt.legend()

roc_curve_plot(y_test, pred_proba[:, 1])

NameError: ignored

In [106]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도:', np.round(accuracy_score(y_target, preds), 4))
print('정밀도:', np.round(precision_score(y_target, preds), 4))
print('재현율:', np.round(recall_score(y_target, preds), 4))

NameError: ignored

In [107]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  f1 = f1_score(y_test, pred)
  # FROC-AUC 추가
  roc_auc = roc_auc(y_test, pred_proba)
  print('오차행렬')
  print(confusion)
  # ROC-AUC print 추가
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
